In [ ]:
import CSV, DataFrames, Plots

base_dir = "./input/train/"
tr1_dir = "defog/"
tr1_fr  = 100
tr2_dir = "tdcsfog/"
tr2_fr  = 128

# simulating the dataLoader
all_files = readdir(base_dir*tr1_dir)

rr = rand(1:length(all_files))
@show rr
# reading in a specific file
df1 = CSV.read(base_dir*tr1_dir*all_files[rr] , DataFrames.DataFrame)
df1.Time ./= tr1_fr;

In [ ]:
events = CSV.read("./input/events.csv", DataFrames.DataFrame)

In [ ]:
nii = size(df1)[1]
sii = rand(1:nii)
aii = sii .+ (1:min(nii-sii,500*tr1_fr))

p = Plots.plot(size=(1300,400)) # layout=(3,1),
color = 2 .* df1.StartHesitation .+ df1.Turn
Plots.scatter!(p, df1.Time[aii], df1.AccV[aii], color=color[aii], label=false,title="AccV",titlefontsize=9,msw=0,ms=1)
Plots.scatter!(p, df1.Time[aii], df1.AccML[aii],color=color[aii], label=false,title="AccML",titlefontsize=9,msw=0,ms=1)
Plots.scatter!(p, df1.Time[aii], df1.AccAP[aii],color=color[aii], label=false,title="AccAP",titlefontsize=9,msw=0,ms=1)

p

In [ ]:
Plots.savefig("need_to_average.pdf")

# Transforming from Freq1 to Freq2

This section is about the transformation of a signal at frequency $\omega_1$ to a signal with frequency $\omega_2$, where frequency means the times a signal is sampled __in a second__.

In [5]:
function transform(d1, fr1::Integer, fr2::Integer)
    
    nd1 = length(d1)
    nd2 = Int(floor(nd1*fr2/fr1))
    λ   = (0:(nd2-1))*fr1/fr2
    ii1 = Int.(floor.(λ))
    λ  -= ii1
    d1  = vcat(deepcopy(d1), d1[end])

    return (1 .- λ) .* d1[1 .+ ii1] + λ .* d1[2 .+ ii1]
end


transform (generic function with 1 method)

In [ ]:

# debug
#t0 = collect(0:1.25:10)
#t,λ = transform(t0,8,10)
#@show t

accV1 = df1.AccV[aii]
time1 = df1.Time[aii]
accV2 = transform(accV1, tr1_fr, tr2_fr)
time2 = transform(time1, tr1_fr, tr2_fr)

p = Plots.scatter(time1, accV1) # df1.AccV[aii])
Plots.scatter!(p, time2, accV2 ,msw=0,ms=2) # 


In [ ]:
Plots.plot!(p,xlim=(1100,1101))

# For the second data-set

In [ ]:
# simulating the dataLoader
all_f2 = readdir(base_dir*tr2_dir)

rr2 = rand(1:length(all_f2))
@show rr2
# reading in a specific file
df2 = CSV.read(base_dir*tr2_dir*all_f2[rr2] , DataFrames.DataFrame)
df2.Time ./= tr2_fr;

In [ ]:
nii = size(df2)[1]
sii = rand(1:nii)
aii = sii .+ (1:min(nii-sii,10*tr2_fr))

p = Plots.plot(layout=(3,1),size=(800,400))
color = 2 .* df2.StartHesitation .+ df2.Turn
Plots.plot!(p[1], df2.Time[aii], df2.AccV[aii], color=color[aii], label=false,title="AccV",titlefontsize=9)
Plots.plot!(p[1], df1.Time[aii], df1.AccV[aii], label=false,title="AccV",titlefontsize=9)

Plots.plot!(p[2], df2.Time[aii], df2.AccML[aii],color=color[aii], label=false,title="AccML",titlefontsize=9)
Plots.plot!(p[2], df1.Time[aii], df1.AccML[aii], label=false,title="AccML",titlefontsize=9)

Plots.plot!(p[3], df2.Time[aii], df2.AccAP[aii],color=color[aii], label=false,title="AccAP",titlefontsize=9)
Plots.plot!(p[3], df1.Time[aii], df1.AccAP[aii], label=false,title="AccAP",titlefontsize=9)


# Your task is to:

1. build a data loader that samples the __events__ (from the "events.csv" file), copies chunks using a number $D$ of _continuous_ samples. The output is assigned as a weights into which classes are present in the series of length $D$.

1. Build a neural network that is capable to read in $D$ values and produce a $K$ outputs, where $K$ is the number of different classifications possible.

1. Evaluate the system for different values of $D$ and different architectures.


In [1]:
# maybe we don't need a struct, 
# ehh 
struct myDataLoader
    sampler
    D
    getNextBatch
end

In [2]:
import CSV, DataFrames, Plots
events = CSV.read("./input/events.csv", DataFrames.DataFrame)


function transform(d1, fr1::Integer, fr2::Integer)
    
    nd1 = length(d1)
    nd2 = Int(floor(nd1*fr2/fr1))
    λ   = (0:(nd2-1))*fr1/fr2
    ii1 = Int.(floor.(λ))
    λ  -= ii1
    d1  = vcat(deepcopy(d1), d1[end])

    return (1 .- λ) .* d1[1 .+ ii1] + λ .* d1[2 .+ ii1]
end


transform (generic function with 1 method)

In [3]:
function loadFile(path)
    CSV.read(path, DataFrames.DataFrame)
end
#load in rows in time interval [first,last]
function loadRecording(file_name, first_index, last_index)
    data = []
    path_tdcs = "./input/train/tdcsfog/"
    path_de = "./input/train/defog/"
    path_notype = "./input/train/notype/"
    frequency = 128
    try 
        data = loadFile(path_tdcs * file_name*".csv")
    catch
        try
            frequency = 100
            data = loadFile(path_de * file_name*".csv")
        catch 
            # it means we have notype
            # do we want to load them in , or what ? because the structure is a bit different, there are no event type annotations.. 
            # data = loadFile(path_notype * file_name*".csv")
        end
    end

    index_start = trunc(Int64,frequency *first_index ) + 2
    index_end = min(trunc(Int64,frequency *last_index ) + 1,size(data)[1])


    (Matrix(data[index_start:index_end,:]),frequency)

end

loadRecording (generic function with 1 method)

In [13]:
#load D samples from events.csv 
function loadSamplesFromEvents(starting_from::Int32, num_d::Int32, event_df::DataFrames.DataFrame )
    num_events = size(event_df)[1]
    ending_with = min(starting_from+num_d-1,num_events) # to avoid "index not found" exception 
    # select events to be loaded 
    # we could also do also just load randomly , if that is desired 
    chosen_events = [event_df[row,:] for row in starting_from:ending_with]
    train_data = []    
    for event in chosen_events
        series_event = loadRecording(event.Id,event.Init,event.Completion)
        push!(train_data,series_event)
        
    end
    
    train_data
end
raw_data = loadSamplesFromEvents(1,10,events)

10-element Vector{Any}:
 ([1103.0 -9.39529972114623 … 1.0 0.0; 1104.0 -9.16660317480122 … 1.0 0.0; … ; 1889.0 -8.5489501493737 … 1.0 0.0; 1890.0 -8.56789013914377 … 1.0 0.0], 128)
 ([1458.0 -8.94870175413821 … 1.0 0.0; 1459.0 -9.05473787136403 … 1.0 0.0; … ; 5270.0 -8.91070800593675 … 1.0 0.0; 5271.0 -8.86710149396174 … 1.0 0.0], 128)
 ([6998.0 -10.2779361750618 … 1.0 0.0; 6999.0 -9.99570845868899 … 1.0 0.0; … ; 7523.0 -9.61131702426238 … 1.0 0.0; 7524.0 -9.17878724878662 … 1.0 0.0], 128)
 ([3597.0 -8.69936143533898 … 1.0 0.0; 3598.0 -8.77127881535382 … 1.0 0.0; … ; 3876.0 -8.60192171420836 … 1.0 0.0; 3877.0 -8.6538965989354 … 1.0 0.0], 128)
 ([3881.0 -9.16232640273608 … 1.0 0.0; 3882.0 -9.13981486072401 … 1.0 0.0; … ; 4079.0 -10.0472318960615 … 1.0 0.0; 4080.0 -10.5986136370634 … 1.0 0.0], 128)
 ([2054.0 -9.58634938862899 … 1.0 0.0; 2055.0 -9.41231919415004 … 1.0 0.0; … ; 2206.0 -9.95826843635929 … 1.0 0.0; 2207.0 -10.0628253297948 … 1.0 0.0], 128)
 ([2510.0 -10.9445153324145 … 1.0 0.

In [5]:
low_freq = [x==100 for (_,x) in raw_data]
raw_data[low_freq]
# I guess data should be transformed.. sometime for training ? anyway.. the first 

Any[]

In [17]:
function transform_batch(batch) 
    # not sure how to do that .. we can , or should only transform the signals.. or we we extend also the labels ?? ehh 
    batch
end

function getNextBatch(data_loader)
    from_index = data_loader.current_index
    chunk_size = data_loader.d
    
    batch = loadSamplesFromEvents(from_index,chunk_size,data_loader.events_dataframe)
    data_loader.current_index += chunk_size
    #maybe there is no need for reset.. mark it as to be removed ? 
    # if (data_loder.current_index > size(data_loader.events_dataframe)[1])
    #     data_loader.current_index = 1
    # end 
    # transform data if neccessary I guess 
    batch = tranform_batch(batch)

    batch
end

getNextBatch (generic function with 1 method)

In [7]:
mutable struct data_loader 
    current_index::Int32
    d::Int32
    getNext
    events_dataframe
end

In [29]:
# create train data 

dataLoader = data_loader(1,10,getNextBatch,events)
dataLoader.getNext(dataLoader)

10-element Vector{Any}:
 ([1103.0 -9.39529972114623 … 1.0 0.0; 1104.0 -9.16660317480122 … 1.0 0.0; … ; 1889.0 -8.5489501493737 … 1.0 0.0; 1890.0 -8.56789013914377 … 1.0 0.0], 128)
 ([1458.0 -8.94870175413821 … 1.0 0.0; 1459.0 -9.05473787136403 … 1.0 0.0; … ; 5270.0 -8.91070800593675 … 1.0 0.0; 5271.0 -8.86710149396174 … 1.0 0.0], 128)
 ([6998.0 -10.2779361750618 … 1.0 0.0; 6999.0 -9.99570845868899 … 1.0 0.0; … ; 7523.0 -9.61131702426238 … 1.0 0.0; 7524.0 -9.17878724878662 … 1.0 0.0], 128)
 ([3597.0 -8.69936143533898 … 1.0 0.0; 3598.0 -8.77127881535382 … 1.0 0.0; … ; 3876.0 -8.60192171420836 … 1.0 0.0; 3877.0 -8.6538965989354 … 1.0 0.0], 128)
 ([3881.0 -9.16232640273608 … 1.0 0.0; 3882.0 -9.13981486072401 … 1.0 0.0; … ; 4079.0 -10.0472318960615 … 1.0 0.0; 4080.0 -10.5986136370634 … 1.0 0.0], 128)
 ([2054.0 -9.58634938862899 … 1.0 0.0; 2055.0 -9.41231919415004 … 1.0 0.0; … ; 2206.0 -9.95826843635929 … 1.0 0.0; 2207.0 -10.0628253297948 … 1.0 0.0], 128)
 ([2510.0 -10.9445153324145 … 1.0 0.

In [32]:
dataLoader.getNext(dataLoader)

10-element Vector{Any}:
 ([2547.0 -8.49477605840235 … 1.0 0.0; 2548.0 -8.43373488137218 … 1.0 0.0; … ; 3640.0 -9.58295280537681 … 1.0 0.0; 3641.0 -9.6381606280407 … 1.0 0.0], 128)
 ([4336.0 -8.7572528813337 … 1.0 0.0; 4337.0 -8.57360442796522 … 1.0 0.0; … ; 4447.0 -8.73567091118528 … 1.0 0.0; 4448.0 -8.71308341254031 … 1.0 0.0], 128)
 ([976.0 -10.2697023739648 … 0.0 1.0; 977.0 -10.4221368734523 … 0.0 1.0; … ; 1168.0 -10.5300521560983 … 0.0 1.0; 1169.0 -10.0656873105244 … 0.0 1.0], 128)
 ([2535.0 -11.3551795848546 … 1.0 0.0; 2536.0 -11.6133956666172 … 1.0 0.0; … ; 2633.0 -11.0483585524381 … 1.0 0.0; 2634.0 -10.8600249441118 … 1.0 0.0], 128)
 ([3951.0 -9.29739441084866 … 1.0 0.0; 3952.0 -9.4475271054468 … 1.0 0.0; … ; 4481.0 -9.40982942699934 … 1.0 0.0; 4482.0 -9.34489392319113 … 1.0 0.0], 128)
 ([1348.0 -9.9681840034427 … 1.0 0.0; 1349.0 -10.4202789376938 … 1.0 0.0; … ; 1469.0 -10.1278391602663 … 1.0 0.0; 1470.0 -10.5649362782316 … 1.0 0.0], 128)
 ([2096.0 -8.42144112275253 … 1.0 0.0; 2